In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# lets say, we have 1000 users that spend around 10E per month on average
np.random.seed(0)
T0 = 1000
t_historical = np.arange(T0)
metric_historical_treatment = np.random.normal(10, 2.5, T0)
metric_historical_control = np.random.normal(10, 2.5, T0)

In [ ]:
#@title plot - before test
plt.figure(figsize=(15, 7))
plt.hist(metric_historical_treatment, color='lightgreen', label='historical treatment', alpha=0.6)
plt.hist(metric_historical_control, color='lightcoral', label='historical control', alpha=0.6)
plt.title('Metric distribution', size=20)
plt.xlabel('metric', size=20)
plt.ylabel('frequency', size=20)
plt.legend(fontsize=20)
plt.show()

In [ ]:
# sample variances, True variance
np.var(metric_historical_control), np.var(metric_historical_treatment), 2.5**2

### Test start

In [ ]:
# assume, the treatment increased the true spend mean from 10E to 12E
np.random.seed(10)
metric_test_treatment = metric_historical_treatment + np.random.normal(2, 1, T0) # (5, 1, T1)
metric_test_control = metric_historical_control + np.random.normal(0, 1, T0)

In [ ]:
#@title plot - after test
plt.figure(figsize=(15, 7))
plt.hist(metric_test_treatment, color='lightgreen', label='TEST treatment', alpha=0.6)
plt.hist(metric_test_control, color='lightcoral', label='TEST control', alpha=0.6)
plt.title('Metric distribution', size=20)
plt.xlabel('metric', size=20)
plt.ylabel('frequency', size=20)
plt.legend(fontsize=20)
plt.show()

In [ ]:
# CUPED - from slides
metric_historical = np.append(metric_historical_treatment, metric_historical_control)
metric_test = np.append(metric_test_treatment, metric_test_control)
theta = np.cov(metric_historical, metric_test)[0, 1] / np.var(metric_historical)

metric_cuped_treatment = metric_test_treatment - theta*metric_historical_treatment + theta*np.mean(metric_historical)
metric_cuped_control = metric_test_control - theta*metric_historical_control + theta*np.mean(metric_historical)

In [ ]:
# @title CUPED impact on metrics
treatment_impact_df = [[round(metric_test_treatment.mean(), 2), round(metric_test_treatment.var(), 2)], 
                    [round(metric_cuped_treatment.mean(), 2), round(metric_cuped_treatment.var(), 2)]]

control_impact_df = [[round(metric_test_control.mean(), 2), round(metric_test_control.var(), 2)], 
                    [round(metric_cuped_control.mean(), 2), round(metric_cuped_control.var(), 2)]]

print('\033[1mTreatment: \033[0m')
print(pd.DataFrame(treatment_impact_df, columns=['mean', 'variance'], index=['original test', 'CUPED adjusted']))

print('\n\033[1mControl: \033[0m')
print(pd.DataFrame(control_impact_df, columns=['mean', 'variance'], index=['original test', 'CUPED adjusted']))

In [ ]:
# @title [Original test vs CUPED distribution] compare variances visually
fig, ax = plt.subplots(1, 2, figsize=(25, 7))
ax[0].hist(metric_test_treatment, color='lightgreen', label='test treatment', bins=10, alpha=0.6)
ax[0].hist(metric_cuped_treatment, color='forestgreen', label='CUPED treatment', bins=10, alpha=0.6)
ax[0].set_title('[Treatment] Metric distribution', size=20)
ax[0].set_xlabel('metric', size=20)
ax[0].set_ylabel('count', size=20)
ax[0].legend(fontsize=20)

ax[1].hist(metric_test_control, color='lightcoral', label='test control', bins=10, alpha=0.6)
ax[1].hist(metric_cuped_control, color='red', label='CUPED control', bins=10, alpha=0.6)
ax[1].set_title('[Control] Metric distribution', size=20)
ax[1].set_xlabel('metric', size=20)
ax[1].set_ylabel('count', size=20)
ax[1].legend(fontsize=20)

plt.show()

In [ ]:
# @title [Treatment vs Control distribution] compare variances visually
fig, ax = plt.subplots(1, 2, figsize=(25, 7))
ax[0].hist(metric_test_treatment, color='lightgreen', label='test treatment', bins=10, alpha=0.6)
ax[0].hist(metric_test_control, color='lightcoral', label='test control', bins=10, alpha=0.6)
ax[0].set_title('[Original test] Metric distribution', size=20)
ax[0].set_xlabel('metric', size=20)
ax[0].set_ylabel('count', size=20)
ax[0].legend(fontsize=20)

ax[1].hist(metric_cuped_treatment, color='forestgreen', label='CUPED treatment', bins=10, alpha=0.6)
ax[1].hist(metric_cuped_control, color='red', label='CUPED control', bins=10, alpha=0.6)
ax[1].set_title('[CUPED] Metric distribution', size=20)
ax[1].set_xlabel('metric', size=20)
ax[1].set_ylabel('count', size=20)
ax[1].legend(fontsize=20)

plt.show()

In [ ]:
# @title [Before/After-test metric distribution] compare variances visually
plt.figure(figsize=(15, 7))
plt.scatter(metric_historical_treatment, metric_test_treatment, color='lightgreen', marker='.', label='TEST treatment')
plt.scatter(metric_historical_control, metric_test_control, color='lightcoral', marker='.', label='TEST control')
plt.scatter(metric_historical_treatment, metric_cuped_treatment, color='forestgreen', marker='.', label='CUPED treatment')
plt.scatter(metric_historical_control, metric_cuped_control, color='red', marker='.', label='CUPED control')
plt.title('Before-After test metric distribution')
plt.xlabel('before test start')
plt.ylabel('after test start')
plt.ylim(0, 20)
plt.legend()
plt.show()

**Just look at how much less variance there is along the y-axis for CUPED, compared to classic TEST.**

------

In [ ]:
# variance difference ratio (from formula for Var(Y_{CUPED}))
print(metric_cuped_treatment.var() / metric_test_treatment.var())
print(metric_cuped_control.var() / metric_test_control.var())
print('\n')
print(1 - np.corrcoef(metric_test_treatment, metric_historical_treatment)[0, 1]**2)
print(1 - np.corrcoef(metric_test_control, metric_historical_control)[0, 1]**2)